## Importing libraries

In [1]:
import numpy as np
from skorch import NeuralNetClassifier
from torch import cuda, nn
from torch.optim import Adam
from torchvision import models
from torchvision.transforms import Compose, ToTensor

from dlordinal.datasets import FGNet
from dlordinal.losses import TriangularCrossEntropyLoss

## Load and preprocess of FGNet dataset

First, we present the configuration parameters for the experimentation and the number of workers for the `DataLoader`, which defines the number of subprocesses to use for data loading. In this specific case, it refers to the images.

In [2]:
optimiser_params = {"lr": 1e-3, "bs": 400, "epochs": 5, "s": 2, "c": 0.2, "beta": 0.5}

workers = 3

Now we use the `FGNet` method to download and preprocess the images.

In [3]:
fgnet_train = FGNet(
    root="./datasets",
    download=True,
    train=True,
    target_transform=np.array,
    transform=Compose([ToTensor()]),
)

fgnet_test = FGNet(
    root="./datasets",
    download=True,
    train=False,
    target_transform=np.array,
    transform=Compose([ToTensor()]),
)

num_classes = len(fgnet_train.classes)
classes = fgnet_train.classes
targets = fgnet_train.targets

# Get CUDA device
device = "cuda" if cuda.is_available() else "cpu"
print(f"Using {device} device")

Files already downloaded and verified
Files already processed and verified
Files already split and verified
Files already downloaded and verified
Files already processed and verified
Files already split and verified
Using cpu device


## Estimator

We are setting up a deep learning model using `PyTorch` and `Skorch`. First, we define the model architecture using ResNet18, a pre-trained convolutional neural network, and customize its fully connected layer to match the number of classes in our classification task. Then we specify the loss function, in this case, a custom Triangular Cross Entropy Loss[1]. Finally, we configure the Skorch estimator, which serves as a bridge between PyTorch and scikit-learn, allowing us to train and evaluate our model seamlessly. We provide the model, loss function, and optimiser details such as the learning rate and number of epochs to the estimator. Additionally, we specify parameters for data loading and processing, like batch size and the number of workers, to optimise training performance.

[1]: Víctor Manuel Vargas, Pedro Antonio Gutiérrez, Javier Barbero-Gómez, and César Hervás-Martínez (2023). *Soft Labelling Based on Triangular Distributions for Ordinal Classification.* Information Fusion, 93, 258--267. doi.org/10.1016/j.inffus.2023.01.003

In [4]:
# Model
model = models.resnet18(weights="IMAGENET1K_V1")
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss function
loss_fn = TriangularCrossEntropyLoss(num_classes=num_classes).to(device)

# Skorch estimator
estimator = NeuralNetClassifier(
    module=model,
    criterion=loss_fn,
    optimizer=Adam,
    lr=optimiser_params["lr"],
    max_epochs=optimiser_params["epochs"],
    train_split=None,
    callbacks=[],
    device=device,
    verbose=0,
    iterator_train__batch_size=optimiser_params["bs"],
    iterator_train__shuffle=True,
    iterator_train__num_workers=workers - 1,
    iterator_train__pin_memory=True,
    iterator_valid__batch_size=optimiser_params["bs"],
    iterator_valid__shuffle=False,
    iterator_valid__num_workers=workers - 1,
    iterator_valid__pin_memory=True,
)

In [5]:
estimator.fit(X=fgnet_train, y=targets)

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(6

In [6]:
train_probs = estimator.predict_proba(fgnet_train)
print(f"Train probabilities = {train_probs=}\n")

test_probs = estimator.predict_proba(fgnet_test)
print(f"Test probabilities = {test_probs=}")

Train probabilities = train_probs=array([[ 2.9335966e+00,  1.6396730e+00, -4.1097212e+00,  4.2018917e-01,
        -1.4565204e+00, -4.3641205e+00],
       [ 8.1791782e+00,  2.8562646e+00, -9.0971680e+00,  1.6580626e-02,
        -3.8278933e+00, -7.0976620e+00],
       [ 9.5499592e+00,  3.5849128e+00, -8.4043884e+00, -4.2262230e-02,
        -5.7516675e+00, -7.0723133e+00],
       ...,
       [ 1.7538257e+00,  1.8087029e-04, -6.2550454e+00,  2.6264958e+00,
         3.1415778e-01, -4.3745127e+00],
       [-6.1797386e-01, -6.9311045e-02, -3.2306197e+00,  1.5903845e+00,
        -5.8013773e-01, -1.2073216e+00],
       [-1.2833995e+00, -2.5462928e-01, -3.4825776e+00,  2.6367762e+00,
         2.4873786e-01, -1.8333929e+00]], dtype=float32)

Test probabilities = test_probs=array([[-0.30555367, -0.06686927, -2.9128444 ,  2.4259052 ,  0.38098484,
        -2.2845333 ],
       [ 6.7591906 ,  4.8296413 , -4.5853643 ,  0.25159836, -5.5993004 ,
        -6.9446783 ],
       [ 1.775908  ,  2.4953337 , -3.